In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

## Load model and test data

In [2]:
experiment_relative_path = r"outputs\2025-08-06_09-53-26_eda_shear_50M"

eval_data_path: Path = Path(r"C:\Users\cervinka\cervinka\GitHub\MathCAS\datasets\dataset_compressible_flow_5M_test.csv")

In [ ]:
# Load m experiment directory #

import torch
from pathlib import Path
from config import load_config, FullConfig
from src.architecture import NeuralNetwork

exp_dir: Path = project_root / Path(experiment_relative_path)

config_path = exp_dir / "config.yaml"

config: FullConfig = load_config(config_path)

## Make predictions on the test data

In [4]:
from src.evaluation import evaluate

evaluate(config, exp_dir, eval_data_path)